In [ ]:
!pip install openai
!pip install pinecone-client
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 25.3 MB/s eta 0:00:00


#1. Load data into a pandas dataframe

In [ ]:
from google.colab import userdata
import openai

openai_api_key = userdata.get('openai-secret')
pinecone_api_key = userdata.get('pinecone-api-key')

In [ ]:
import fitz
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    texts = [page.get_text() for page in doc]
    doc.close()
    return texts

In [ ]:
from openai import OpenAI

def get_embeddings(text):
    client = OpenAI(api_key=openai_api_key)
    response = client.embeddings.create(input=text,model="text-embedding-ada-002")
    return response.data[0].embedding

In [ ]:
import pandas as pd
import uuid
import json

def create_dataframe(texts):
    data = {
        "id": [],
        "embeddings_vector": [],
        "metadata": []
    }

    for i, text in enumerate(texts):
        data["id"].append(str(uuid.uuid4()))
        data["embeddings_vector"].append(get_embeddings(text))
        data["metadata"].append(json.dumps({"chunk": i, "page": i + 1, "text": text}))

    return pd.DataFrame(data)

In [ ]:
texts = extract_text_from_pdf('/content/original.pdf')

df = create_dataframe(texts)
df.head()

,id,embeddings_vector,metadata
0,f8123984-9bda-4272-8924-22b285510e22,"[0.001079675043001771, -0.02435639686882496, 0...","{""chunk"": 0, ""page"": 1, ""text"": ""E-Book series..."
1,a04c9104-6767-494a-ac30-f1216478605a,"[0.0026940391398966312, 0.0057278964668512344,...","{""chunk"": 1, ""page"": 2, ""text"": ""2\nThe Develo..."
2,5b8caa63-8ce6-411b-adc6-001700450ca7,"[-0.0008800688083283603, 0.005942094139754772,...","{""chunk"": 2, ""page"": 3, ""text"": ""3\nThe Develo..."
3,9e397e5e-168e-496d-a882-5d24cb4c14ba,"[0.0003556290757842362, 0.0039048835169523954,...","{""chunk"": 3, ""page"": 4, ""text"": ""4\nThe Develo..."
4,af1236cb-6985-41cf-a04d-5956de282bb3,"[-5.671835970133543e-05, -0.003213367657735944...","{""chunk"": 4, ""page"": 5, ""text"": ""5\nThe Develo..."


#2. Upsert embeddings into pinecone index

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index("tech16llm01")
index

In [ ]:
import json

vectors = []
for i, row in df.iterrows():
  vectors.append({
      "id": row['id'],
      "values": row['embeddings_vector'],
      "metadata": json.loads(row['metadata'])
  });

index.upsert(vectors, namespace='azure-developers')

{'upserted_count': 140}

#3. RAG

In [ ]:
query = "Mention some Azure services to implement a distributed architecture."

query_embeddings = get_embeddings(query)
query_result = index.query(vector=query_embeddings, top_k=3, include_metadata=True, namespace = 'azure-developers')
query_result

{'matches': [{'id': 'ff4d82da-3bc6-4a66-b081-919a1abad6bd',
              'metadata': {'chunk': 58.0,
                           'page': 59.0,
                           'text': '59\n'
                                   'The Developer’s Guide to Azure\n'
                                   'Geo-distributed applications\n'
                                   'The geo-distributed approach has two main '
                                   'goals:\n'
                                   '• Provide a seamless experience to its '
                                   'users \n'
                                   'anywhere in the world.\n'
                                   '• Provide high availability and maximum '
                                   'uptime.\n'
                                   'In Figure 3.4, you can see a possible '
                                   'architecture for a \n'
                                   'geo-distributed application. AKS is placed '
                         

In [ ]:
contexts = [item['metadata']['text'] for item in query_result['matches']]
augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query
print(augmented_query)

59
The Developer’s Guide to Azure
Geo-distributed applications
The geo-distributed approach has two main goals:
• Provide a seamless experience to its users 
anywhere in the world.
• Provide high availability and maximum uptime.
In Figure 3.4, you can see a possible architecture for a 
geo-distributed application. AKS is placed in several 
different regions and network traffic is controlled 
with Traffic Manager to direct the user to the closest 
available AKS instance to complete the request.
Figure 3.4: Geo-distributed application design in Azure
The Microsoft Azure infrastructure is composed of 
more than 160 datacenters in more than 60 different 
regions. With services such as Azure Front Door or 
Traffic Manager, you can create globally distributed 
applications. This way, you can ensure that applications 
are up and running even with issues in one of the 
datacenters. As the application is hosted in multiple 
locations, users are routed to the closest instance, 
providing minimum

In [ ]:
# system message to 'prime' the model
primer = f"""You are Q&A bot. A highly intelligent system that answers
user questions based on the information provided by the user above
each question. If the information can not be found in the information
provided by the user you truthfully say "I don't know".
"""

from openai import OpenAI
client = OpenAI(api_key=openai_api_key)

response = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": primer},
    {"role": "user", "content": augmented_query}
  ]
)

In [ ]:
from IPython.display import Markdown

display(Markdown(response.choices[0].message.content))

Some Azure services to implement a distributed architecture mentioned in the text include Azure Front Door, Traffic Manager, and Azure Kubernetes Service (AKS). You can also use Azure Cosmos DB for low-latency data access from anywhere in the world, and Azure Arc for easier management of infrastructure.